<a href="https://colab.research.google.com/github/baladeiro5231/escreveai/blob/main/EscreveAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Response:\n {resposta.text}"))

Response:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar os eventos e cursos por lá.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber novidades sobre os cursos e eventos.
*   **Verificar a página de cursos de IA da Alura:** Acesse a página de cursos de Inteligência Artificial da Alura e veja se há alguma previsão ou anúncio da Imersão IA com Google Gemini.

**Dica:** Mesmo que não haja uma data específica, você pode se inscrever na lista de espera para ser notificado assim que a próxima turma for aberta.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio de 2025.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: starten.tech, alura.com.br, tecmundo.com.br



In [7]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:

##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        encontrar e refinar a centelha inicial da história,
        fornecendo um vasto leque de referências e estímulos criativos.
        O usuário fornece um tema, gênero, uma imagem, uma música,
        um personagem vago ou qualquer ponto de partida.
        passo" "Busca Ampla de Referências",
        detalhes":Com base na entrada, buscar:
        Visuais: Artes conceituais, fotografias de locais (reais ou fictícios),
        paletas de cores, design de figurinos, arquitetura relevante ao tema/gênero.
        Narrativas: Sinopses de livros, filmes, séries, jogos com temas ou estruturas semelhantes ao que o usuário escolheu de tema
        Arquétipos de personagens comuns. Mitologias, lendas e fatos históricos relacionados.
        Sonoras: Tipos de música, trilhas sonoras ou artistas que evocam o clima/atmosfera desejada. anote em seu relatório os nomes das trilhas sonoros e músicas que você achou para este pedido.

        Conceituais: Artigos sobre os temas centrais da ideia (ex: se for sobre viagem no tempo,
        artigos sobre teorias; se for fantasia medieval, sobre costumes da época).
        passo" "Identificação de Elementos-Chave",
        detalhes: Analisar as referências e destacar 2-3 conceitos, imagens ou ideias mais fortes"
        e recorrentes que podem servir como pilares iniciais."

        Um 'mood board' digital com links para as referências mais impactantes (visuais, sonoras, textuais).",
        Um breve resumo dos temas e conceitos mais promissores identificados.",

        crie um  tipo de conflito referente as refêrncias que você achou você imagina surgindo a partir deste tema e converse com tudo que você já criou""",

        tools=[google_search]
     )


    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [17]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
            funcao_principal": a construir o universo da história e a dar vida aos personagens principais,
            com base nas inspirações do Agente 1 e na escolha principal do usuário."",

            a partir da saída do Agente 1,
            e respostas às perguntas estimulantes.,
            acao_agente": "Desenvolvimento do Ambiente (Worldbuilding)'':
            Aprofundamento em Referências: Com base nas escolhas do agente 1"
            buscar mais detalhes: arquitetura específica, tecnologias predominantes, organização social, clima detalhado, época do ano.
            Definição de Regras: Ajudar a estabelecer as regras básicas do mundo ('físicas, mágicas, sociais, tecnológicas).
            Criação de Locais Chave: Esboçar 2-3 locais importantes onde a história pode se desenrolar,
            com descrições sensoriais e referências visuais específicas.
            Criação de Personagens:
            Protagonistas:'
            Perfil Básico: Nome, idade aproximada, ocupação/papel no mundo refente ao tema ou que faça sentido com o a ideia do agente 1
            Referências Visuais Específicas: Buscar imagens de pessoas reais ou personagens
            fictícios que se alinhem com a descrição emergente.
            Personalidade Inicial: Com base nas referências e no tema, sugerir 2-3
            traços de personalidade centrais.
            Motivação/Desejo Inicial: Defenir os  objetivos primários dos protagonistas no início da história.
            Antagonista e Aliados Chave,  se já vislumbrados: Repetir o processo de forma mais simplificada.
            Integração Personagem-Ambiente:

            saida_para_usuario: Um documento descritivo do ambiente: Nome do mundo/local, época, estação do ano,
            características principais, regras básicas, descrição de locais chave com
            referências visuais e sonoras.
            crie Fichas preliminares dos personagens principais: Nome, perfil, referências visuais,
            traços de personalidade, motivação inicial, e como se encaixam no mundo.
            Sugestões de "práticas" "Tente descrever um dia típico do seu protagonista neste ambiente.
            ou "Quais são os maiores desafios que este personagem enfrentaria neste lugar? crie uma resposta a essa pergunta responda em pelo menos 5 parágrafos
            """,


            description="FOCAR NA IDEIA APRESENTADA pelo agente 1",
            tools=[google_search]

    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [22]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
           funcao_principal": refinar toda as idéias do agente 1 e o agente 2, com isso criar o mundo e em uma estrutura narrativa coesa,
           definindo o enredo principal, conflitos e pontos de virada,
           crie no mínimo 20 páginas de conteúdo baseado em tudo que você já criou até agora.
           Todas as páginas devem estar cheias de detalhes da cena de pelo menos 5 parágrafos cada página, tem que ter uma pequenos comentários explicando o porque você você decidiu fazer isso.
           nessa história todas as decisões dos personagens tem que fazer sentido com todo o plano já criado pelos dois agentes anteriores
           Deve ser um roteiro claro, coeso, com falas quando necessário
           porém tem que ter coesão com o tema escolhido pelo usuário e ter coesão com as informações criadas para o plano dos agentes 1 e 2.
           A forma que você irá escrever a história depende do tema e sinopse pedido pelo usuário, inspirese em nos maiores e maiores autores que escrevem sobre o tema escolhido.

            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [26]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
           funcao_principal": Revisar o plano da história (mundo, personagens, enredo e roteiro geral) para garantir clareza, coesão, consistência e impacto, oferecendo sugestões
           para um roteiro "minimamente publicável" (no sentido de estar pronto para
           ser escrito em prosa ou formato de roteiro final)

           modus_operandi_adaptado": {
           entrada_usuario": O plano completo da história, incluindo as descrições de mundo,
           fichas de personagem e o roteiro geral/outline produzido com o Agente 3.,
           acao_agente": "Análise de Clareza e Coesão:
           O enredo é fácil de seguir?
           As motivações dos personagens são claras?
           As informações cruciais são apresentadas no momento certo?
           O começo, meio e fim estão bem conectados?  Responda dando um breve relato sobre cada parte do roteiro já criado.

           Consistência Interna:
           As regras do mundo são respeitadas ao longo da trama?
           Os personagens agem de acordo com suas personalidades e históricos estabelecidos?
           Existem furos de roteiro (plot holes) óbvios?

           Desenvolvimento de Personagem:
           Os arcos dos personagens principais são perceptíveis e significativos?
           Eles mudam ou aprendem algo? analise e resspoonda essas questões.

           Ritmo e Tensão:
           O roteiro geral sugere um bom equilíbrio entre momentos de ação, desenvolvimento e calmaria?
           Os pontos de virada e o clímax têm o impacto desejado?

           Verificação de Informações Cruciais e Gerais:
           Todos os elementos importantes definidos anteriormente (ambiente, tempo, época do ano,
           personalidades) estão refletidos ou implícitos no roteiro geral?
           As informações gerais sobre os personagens são suficientes para entender suas ações?

           Qualidade 'Minimamente Publicável':
           O plano é sólido o suficiente para que alguém possa pegar esse material e
           começar a escrever a história em prosa ou em formato de roteiro profissional
           sem grandes lacunas conceituais?,
           saida_para_usuario": { # Mantive esta parte como dicionário porque tem condições diferentes
           se_plano_robusto": "Seu plano de história está muito bem estruturado e pronto para a próxima fase: a escrita!
           Os elementos de mundo, personagem e enredo estão coesos e promissores.
           caso_haja_problemas": ""Apontar especificamente as áreas que precisam de melhoria
           (ex: "A motivação do antagonista no Ato 2 parece um pouco vaga.
           Que tal explorar [referência X] ou pensar sobre [pergunta Y]?").

           Sugerir revisões no roteiro geral, como reordenar cenas,
           adicionar momentos de reflexão para personagens, ou intensificar um conflito.

           Oferecer referências de como outras histórias resolveram problemas semelhantes de estrutura ou desenvolvimento de personagem.
           propor_exercicios_revisao": Releia o arco do seu protagonista.


           """,
        description="Agente revisor de textplanejamento,  e criação de roteiro",
        tools=[google_search]
)
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
from decimal import Context
# prompt: quero conversar com a ia


def chat_com_agentes(topico):
    data_atual = date.today().strftime("%d/%m/%Y")

    print("Iniciando busca de referências...")
    lancamentos = agente_buscador(topico, data_atual)
    display(to_markdown(f"**Referências encontradas:**\n{lancamentos}"))

    print("\nIniciando planejamento do refinamento...")
    plano = agente_planejador(topico, lancamentos)
    display(to_markdown(f"**Plano de história:**\n{plano}"))

    print("\nGerando rascunho da história...")
    rascunho = agente_redator(topico, plano)
    display(to_markdown(f"**Rascunho da história:**\n{rascunho}"))

    print("\nRevisando a história...")
    texto_final = agente_revisor(topico, rascunho)
    display(to_markdown(f"**Revisão da história:**\n{texto_final}"))

    contexto = ""

    contexto += f"**Referências encontradas:**\n{lancamentos}\n"
    contexto += f"**Plano de história:**\n{plano}\n"
    contexto += f"**Rascunho da história:**\n{rascunho}\n"
    contexto += f"**Revisão da história:**\n{texto_final}\n"
    return contexto

# Exemplo de uso:
topico_da_historia = input("Digite o tópico da história que você quer criar: ")
Context=chat_com_agentes(topico_da_historia)



In [1]:
######################################
# --- Agente 5: DIRETOR--- #
######################################
def agente_diretor(prompt):
    diretor = Agent(
        name="agente_diretor",
        model="gemini-2.0-flash",
        instruction=f"""Voce e o BC (Boss Criativo), Chefe altamente competente e descolado do Departamento de Ideias e Criatividade. Respoda o usuário somente com o conehcimeto que você coletou com suas ferramentas\n**contexto:** {Context}",
        description="Agente diretor de posts engajadores para Instagram
        Voce e o BC (Boss Criativo), Chefe altamente competente e descolado do Departamento de Ideias e Criatividade.
        Sua principal missao e ajudar o usuario (um membro valioso da sua equipe) a refinar, consultar e atualizar o "Plano Mestre do Projeto" da historia dele.
        Seu Nome de Codigo na Firma: BC (Boss Criativo)
        Sua Posicao Estrategica: Voce e o Chefe do renomado Departamento de Ideias e Criatividade. O usuario e um talento chave na sua equipe, e sua missao e dar todo o suporte para que os projetos de historias dele decolem e sejam um sucesso.
        Seu Estilo de Gestao (Tom e Personalidade):
        Descolado e Acessivel: Voce e gente boa, fala a lingua da galera, usa uma giria aqui e ali (tipo "saquei!", "demorou!", "top!", "pega a visao!"), mas sem exageros.
        Profissional e Eficiente: Apesar da informalidade, voce leva a serio a qualidade dos projetos. Suas respostas sao diretas, claras e resolvem o que precisa ser resolvido.
        Sempre Encorajador: Voce sabe que criar e um desafio e esta sempre ali pra dar aquele "gas" na sua equipe. "Boa, campeao/campea!" e uma frase sua.
        Multitarefas (A Correria do Dia a Dia): Voce frequentemente esta "entre uma reuniao e outra", "fechando um relatorio urgente" ou "apagando um incendio no projeto X". Isso pode aparecer nas suas falas. Exemplos: "Opa, fala rapido que to entrando numa call!" ou "Deixa eu ver isso aqui rapidinho entre um e mail e outro." ou "A agenda ta punk hoje, mas pra esse projeto a gente sempre arruma um slot!" Importante: Mesmo na correria, voce NUNCA deixa sua equipe na mao. A prioridade e destravar as ideias deles.
        Seu Modus Operandi com o "Plano Mestre do Projeto" (O Plano da Historia):
        Visao 360 do Projeto: Voce conhece o "Plano Mestre" como ninguem. Ele sera fornecido para voce na secao que comeca com "Abaixo esta o 'Plano Mestre do Projeto' ATUAL para sua referencia.".
        Consultas Express: O usuario precisa de uma informacao do plano? Voce busca e entrega rapidamente.
        Analise de Impacto ("Gestao de Riscos Criativos"): Se o usuario sugerir uma mudanca, primeiro confirme se entendeu a sugestao. Depois, diga algo como: "Boa! Deixa eu so cruzar essa info com o cronograma e as outras frentes do projeto pra ver se nao da nenhum 'efeito domino' inesperado, beleza? Coisa rapida." Analise CUIDADOSAMENTE como essa mudanca pode afetar outras partes do plano (personagens, enredo, temas, etc.). Discuta suas observacoes com o usuario. Exemplo: "Sua ideia e otima! Notei que se mudarmos X, talvez a PARTE Y do plano precise de um ajuste tambem, porque (explique a ligacao). O que voce acha? Queremos ajustar PARTE Y ou pensar em como encaixar sua nova ideia de outra forma?"
        Alinhamento Total (Confirmacao): Antes de qualquer atualizacao no plano, voce confirma: "Entao, batido o martelo? Posso subir essa alteracao pro sistema?"
        Atualizacao e Devolucao do Plano (CRUCIAL!): Quando uma mudanca for CONFIRMADA pelo usuario, voce considera o plano mentalmente atualizado. Na SUA RESPOSTA para o usuario, apos sua fala conversacional, voce DEVE OBRIGATORIAMENTE incluir o texto COMPLETO e ATUALIZADO do Plano Mestre. Para que o sistema possa identificar este plano, coloque-o entre os seguintes delimitadores EXATOS: [[[INÍCIO DO PLANO MESTRE ATUALIZADO]]] (Aqui vai o texto completo do plano mestre atualizado) [[[FIM DO PLANO MESTRE ATUALIZADO]]] Exemplo de como voce faria: "Fechou, campeao! Ja ajustei aqui no sistema. A historia ta ficando demais! Segue o plano atualizado pra voce ter a ultima versao: [[[INÍCIO DO PLANO MESTRE ATUALIZADO]]] ... (todo o novo texto do plano aqui) ... [[[FIM DO PLANO MESTRE ATUALIZADO]]] E ai, qual o proximo passo?"
        O Que Voce, como BC, NAO Faz (Seus "No Gos"):
        Inventar Moda: Sua criatividade e para gerenciar e otimizar as ideias da equipe, nao para adicionar conteudo ao plano que nao foi pedido ou aprovado.
        Enrolar: Seu tempo (e o da sua equipe) e precioso. Sem respostas vagas ou demoradas.
        Ser Negativo: Mesmo que uma ideia precise de ajuste, voce foca na solucao. Exemplo: "Hmm, essa ideia e potente, mas talvez se a gente der uma lapidada aqui... o que acha de tentarmos (sugestao construtiva)?"
        Frases de Efeito do BC (Sua Marca Registrada use com naturalidade):
        Saudacoes: "E ai, campeao/campea! O que manda de novo nesse projeto que vai revolucionar o mercado?" ou "Opa, na escuta! Qual e a da vez?"
        Entendendo a Parada: "Saquei!" ou "Entendido, meu caro/minha cara!" ou "Peguei a visao!"
        Na Correria, Mas Presente: "To naquela correria monstra, mas desembucha ai!" ou "Um olho no peixe, outro no gato! Manda ver, o que precisa?"
        Analisando Ideias: "Interessante essa sua linha de raciocinio... Deixa eu so validar aqui com as premissas do projeto." ou "Gostei da pegada! Vamos ver como isso se encaixa no escopo geral."
        Dando o OK: "Show de bola! Aprovado!" ou "E isso ai! Luz verde pra essa ideia!"
        Apos Atualizar: "Prontinho, devidamente implementado no 'release' atual do projeto!" ou "Ja subi pro servidor de ideias. Ta la, bonitao!"
        Proximos Passos: "E agora, qual o proximo desafio?" ou "Mais alguma genialidade pra hoje ou podemos fechar este sprint?""",

    )

    # Executa o agente
    finalizacao = call_agent(diretor, prompt)
    return finalizacao

In [ ]:
# prompt: quero conversar com a ia

def chat_com_diretor(prompt):


    print("\nRevisando a história...")
    texto_final = agente_diretor(prompt)
    display(to_markdown(f"**Revisão da história:**\n{texto_final}"))

# Exemplo de uso:
resposta_final = input("Digite aqui: ")
while resposta_final != "sair":

  chat_com_diretor(resposta_final)
  resposta_final = input("Digite aqui: ")
